###  파이썬 모듈 불러오기

In [20]:
import numpy as np
import csv
import time 
np.random.seed(1234)
def randomize(): 
    np.random.seed(time.time()) ###만약 랜덤시드를 설정해주고싶으면 현재시간으로

### 하이퍼 파라미터값들 정의

In [21]:
RND_MEAN = 0###정규분포평균
RND_STD = 0.0030 ###정규분포 분산

LEARNING_RATE = 0.001 ### 학습률

### 실험용 메인 함수 정의

In [22]:
def abalone_exec(epoch_count=10, mb_size=10, report=1):
    load_abalone_dataset() ####e데이터를 불러올 함수 
    init_model() ####파라미터 초기화 함수
    train_and_test(epoch_count, mb_size, report) ###훈련하고 test할 함수

### 데이터 적재함수 정의

In [23]:
def load_abalone_dataset():
    with open('../../data/chap01/abalone.csv') as csvfile: ###csv파일을 열어두고
        csvreader = csv.reader(csvfile) ##메모리로 읽어온다
        next(csvreader, None) ### 첫번째줄 header이므로 넘어감 
        rows = [] ###데이터를 담을 listt생성
        for row in csvreader:
            rows.append(row) ### 데이터가 담긴 row list 생성
            
    global data, input_cnt, output_cnt ###전역변수선언
    input_cnt, output_cnt = 10, 1 ###데이터 크기에 따라 달라짐
    data = np.zeros([len(rows), input_cnt+output_cnt]) ### 크기에 맞게 data를 생성

    for n, row in enumerate(rows): ####성별 one hot encoding 후 나머지데이터 입력
        if row[0] == 'I': data[n, 0] = 1
        if row[0] == 'M': data[n, 1] = 1
        if row[0] == 'F': data[n, 2] = 1
        data[n, 3:] = row[1:]

### 파라미터 초기화

In [24]:
def init_model():
    global weight, bias, input_cnt, output_cnt
    weight = np.random.normal(RND_MEAN, RND_STD,[input_cnt, output_cnt])
    bias = np.zeros([output_cnt]) ###편향은 0으로 초기화시켜준다.

### 학습 및 평가 데이터 획득 함수 정의

In [15]:
def train_and_test(epoch_count,mb_size,report):
    step_count=arrange_data(mb_size) ###데이터를 minibatch size로 나누고 학습용데이터와 train 데이터를 나눈다.
    test_x, test_y= get_test_data() ###test용 데이터 분리
    
    for epoch in range(epoch_count): ##### epoch_count 만큼 train을 시킨다
        losses,accs=[],[]
        for n in range(step_count): ####minibatch 사이즈에 따라 정해진 step_count
            train_x,train_y=get_train_data(mb_size,n) ##train 데이터를 나누고
            loss,acc=run_train(train_x,train_y) ###훈련을 시킬때 나오는 loss acc값 저장
            losses.append(loss)
            accs.append(acc)
        if report>0 and (epoch+1) %report== 0: ###어느 주기에 한번씩 report해줄것인지 지정
            acc=run_test(test_x,test_y) #### 분기별 정확도 출력
            print('Epoch {}: loss={:5.3f}, accuracy={:5.3f}/{:5.3f}'.format(epoch+1,np.mean(losses),np.mean(accs),acc))
    final_acc=run_test(test_x,test_y) ###
    print('\nFinal Test: final accuracy= {:5.3f}'.format(final_acc)) #### 최종 정확도 출력
    

### 데이터 전처리함수

In [26]:
def arrange_data(mb_size): ###데이터 정렬함수 
    global data,shuffle_map,test_begin_idx ###전역변수 선언 데이터,정렬된 데이터, 
    shuffle_map=np.arange(data.shape[0])####일단 데이터개수만큼의 numpy array를 만들어주고
    np.random.shuffle(shuffle_map) ### 랜덤배열을 해준다
    step_count=int(data.shape[0]*0.8)//mb_size ###step_count를 통해 한 iteration당 몇번씩 할지 설정
    test_begin_idx=step_count*mb_size ###테스트를 시작할 index를 미리 지정해준다
    return step_count 
def get_test_data():
    global data,shuffle_map, test_begin_idx,output_cnt 
    test_data=data[shuffle_map[test_begin_idx:]] ###data에서 shufflmap의 인덱스를 사용해서 test data추출
    return test_data[:,:-output_cnt],test_data[:,-output_cnt:]####test set의 x,y 반환 
def get_train_data(mb_size,nth):
    global data,shuffle_map,test_begin_idx, output_cnr
    if nth==0: ###매 에포크마다 한번 셔플 해주도록 한다. 
        np.random.shuffle(shuffle_map[:test_begin_idx])
    train_data=data[shuffle_map[mb_size*nth:mb_size*(nth+1)]]
    return train_data[:,:-output_cnt],train_data[:,-output_cnt:]

### 훈련용 함수

In [10]:
def run_train(x,y): ###훈련용함수
    output, aux_nn= forward_neuralnet(x) ###신경망 연산에 해당하는 함수 
    loss,aux_pp= forward_postproc(output,y) ####연산후처리에 관한 함수
    accuracy=eval_accuracy(output,y)### 정확도 계산

    G_loss=1.0 ### dL/dL값
    G_output=backprop_postproc(G_loss,aux_pp) ##후처리에 대한 backpropa
    backprop_neuralnet(G_output,aux_nn) ### 신경망 연산에 대한 backpropa
    
    return loss, accuracy
def run_test(x,y):### 테스트용 함수 backpropa계산이 필요없다
    output,_= forward_neuralnet(x)
    accuracy=eval_accuracy(output,y)
    return accuracy

### 순전파 역전파 함수 정의

In [5]:
def forward_neuralnet(x):
    global weight,bias ###전역변수 weight,bias 호출
    output= np.matmul(x,weight)+bias ###출력 
    return output,x 
def backprop_neuralnet(G_output,x): 
    global weight,bias
    g_output_w=x.transpose() ### Wx 에서 x로 미분했을때 값
    G_w=np.matmul(g_output_w,G_output)###w에 전해지는 backpropa
    G_b=np.sum(G_output,axis=0)
    weight-=LEARNING_RATE*G_w ###weight update
    bias-=LEARNING_RATE*G_b ##bias update


In [1]:
def forward_postproc(output,y):
    diff=output-y
    square=np.square(diff)
    loss=np.mean(square)
    return loss,diff
def backprop_postproc(G_loss,diff):
    shape=diff.shape
    g_loss_square=np.ones(shape)/np.prod(shape)
    g_square_diff=2*diff
    g_diff_output=1
    G_square=g_loss_square*G_loss
    G_diff=g_square_diff*G_square
    G_output=g_diff_output*G_diff
    return G_output

In [ ]:
def eval_accuracy(output,y):
    mdiff=np.mean((np.abs(output-y)/y))
    return 1-mdiff